# AI DEMO - Summit -Tensorflow Dance Move Inference

This note book should be able to run on the OCP4 Demo Cluster by simply selecting "Cell" and then "Run All" at the top of the page.

**BEFORE**: Run the Flask v5 serving : `./v5_Gesture_Recognition_Serving.py` from within the **virtual environment**

Running this notebook will:
* Query model for predictions

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import json
import time
import os

import itertools
import joblib

import s3fs

import numpy as np
import pandas as pd
import scipy
import scipy.stats

import matplotlib as mpl
import matplotlib.pylab as plt

from pathlib import Path
from datetime import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA

import tensorflow as tf
from tensorflow import keras

---

---

### Data Loading

In [7]:
data_dance_raw = json.loads(Path('data/dance/data_dance_raw.json').read_text())
data_cross_raw = json.loads(Path('data/dance/data_cross_raw.json').read_text())

## Prediction

### Local

In [8]:
import requests

data = {
    "instances": data_dance_raw['floss'][-20:] + data_dance_raw['fever'][-20:]
}
json_response = requests.post("http://localhost:5000/model/predict", json=data, headers={"Host": 'localhost', "Content-Type":"application/json"})

print(json.dumps(json_response.json(), indent=4))

{
    "payload": [
        {
            "candidate": "floss",
            "candidate_score": 0.9946881532669067,
            "predictions": {
                "draw-circle": 7.074011705299199e-07,
                "draw-cross": 0.00022650485334452242,
                "fever": 0.005001303739845753,
                "floss": 0.9946881532669067,
                "roll": 8.151080692186952e-05,
                "shake": 1.8248039168611285e-06
            }
        },
        {
            "candidate": "floss",
            "candidate_score": 0.9999982118606567,
            "predictions": {
                "draw-circle": 1.4799952577490214e-14,
                "draw-cross": 1.6754064802171342e-08,
                "fever": 1.7991450249610352e-06,
                "floss": 0.9999982118606567,
                "roll": 1.891824058519731e-10,
                "shake": 1.889864087445403e-08
            }
        },
        {
            "candidate": "floss",
            "candidate_score": 1.0,
           

In [9]:
for d in data['instances']:
    json_response = requests.post(
        "http://localhost:5000/model/predict",
        json={
            'instances': [d]
        },
        headers={"Host": 'localhost', "Content-Type":"application/json"}
    )

    print(json.dumps(json_response.json(), indent=4))

{
    "payload": [
        {
            "candidate": "floss",
            "candidate_score": 0.9946881532669067,
            "predictions": {
                "draw-circle": 7.074025347719726e-07,
                "draw-cross": 0.00022650528990197927,
                "fever": 0.005001315381377935,
                "floss": 0.9946881532669067,
                "roll": 8.151087968144566e-05,
                "shake": 1.8248056221636944e-06
            }
        }
    ],
    "total": 1
}
{
    "payload": [
        {
            "candidate": "floss",
            "candidate_score": 0.9999982118606567,
            "predictions": {
                "draw-circle": 1.480006438583925e-14,
                "draw-cross": 1.675412875101756e-08,
                "fever": 1.7991621916735312e-06,
                "floss": 0.9999982118606567,
                "roll": 1.891834883194221e-10,
                "shake": 1.8898784759358023e-08
            }
        }
    ],
    "total": 1
}
{
    "payload": [
        

{
    "payload": [
        {
            "candidate": "floss",
            "candidate_score": 1.0,
            "predictions": {
                "draw-circle": 0.0,
                "draw-cross": 2.9293044433467744e-36,
                "fever": 0.0,
                "floss": 1.0,
                "roll": 0.0,
                "shake": 8.591098649769312e-22
            }
        }
    ],
    "total": 1
}
{
    "payload": {
        "candidate": null,
        "candidate_score": null,
        "predictions": null
    },
    "total": null
}
{
    "payload": [
        {
            "candidate": "roll",
            "candidate_score": 0.8203741312026978,
            "predictions": {
                "draw-circle": 0.029343917965888977,
                "draw-cross": 3.505396806158245e-10,
                "fever": 0.0003715181373991072,
                "floss": 1.3656936914685502e-08,
                "roll": 0.8203741312026978,
                "shake": 0.14991046488285065
            }
        }
    ],

{
    "payload": [
        {
            "candidate": "fever",
            "candidate_score": 0.9896512031555176,
            "predictions": {
                "draw-circle": 1.0642022374440785e-07,
                "draw-cross": 2.371413954094237e-09,
                "fever": 0.9896512031555176,
                "floss": 1.4821232241502003e-07,
                "roll": 3.270409365541127e-08,
                "shake": 0.01034846343100071
            }
        }
    ],
    "total": 1
}
{
    "payload": {
        "candidate": null,
        "candidate_score": null,
        "predictions": null
    },
    "total": null
}
{
    "payload": [
        {
            "candidate": "fever",
            "candidate_score": 0.9987900853157043,
            "predictions": {
                "draw-circle": 1.553169681756117e-08,
                "draw-cross": 1.0978459807908791e-10,
                "fever": 0.9987900853157043,
                "floss": 2.618965481815394e-05,
                "roll": 9.42203204346

### Threaded

In [10]:
import concurrent


def query_model_prediction(d, timeout):
    return requests.post(
        "http://localhost:5000/model/predict",
        json={
            'instances': [d]
        },
        headers={"Host": 'localhost', "Content-Type":"application/json"},
        timeout=timeout
    )

    
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Start the load operations and mark each future with its URL
    future_to_prediction = {
        executor.submit(query_model_prediction, d, 60): d for d in data['instances']
    }
    for future in concurrent.futures.as_completed(future_to_prediction):
        input_gesture = future_to_prediction[future]
        try:
            json_response = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (input_gesture, exc))
            
        print(json.dumps(json_response.json(), indent=4))

{
    "payload": [
        {
            "candidate": "floss",
            "candidate_score": 0.9946881532669067,
            "predictions": {
                "draw-circle": 7.074025347719726e-07,
                "draw-cross": 0.00022650528990197927,
                "fever": 0.005001315381377935,
                "floss": 0.9946881532669067,
                "roll": 8.151087968144566e-05,
                "shake": 1.8248056221636944e-06
            }
        }
    ],
    "total": 1
}
{
    "payload": [
        {
            "candidate": "floss",
            "candidate_score": 0.9999982118606567,
            "predictions": {
                "draw-circle": 1.480006438583925e-14,
                "draw-cross": 1.675412875101756e-08,
                "fever": 1.7991621916735312e-06,
                "floss": 0.9999982118606567,
                "roll": 1.891834883194221e-10,
                "shake": 1.8898784759358023e-08
            }
        }
    ],
    "total": 1
}
{
    "payload": [
        

{
    "payload": [
        {
            "candidate": "floss",
            "candidate_score": 1.0,
            "predictions": {
                "draw-circle": 0.0,
                "draw-cross": 1.7825232280828612e-22,
                "fever": 6.256347703088995e-35,
                "floss": 1.0,
                "roll": 2.266059645097855e-38,
                "shake": 0.0
            }
        }
    ],
    "total": 1
}
{
    "payload": [
        {
            "candidate": "floss",
            "candidate_score": 1.0,
            "predictions": {
                "draw-circle": 0.0,
                "draw-cross": 2.9293044433467744e-36,
                "fever": 0.0,
                "floss": 1.0,
                "roll": 0.0,
                "shake": 8.591098649769312e-22
            }
        }
    ],
    "total": 1
}
{
    "payload": [
        {
            "candidate": "roll",
            "candidate_score": 0.8203741312026978,
            "predictions": {
                "draw-circle": 0.029

{
    "payload": [
        {
            "candidate": "fever",
            "candidate_score": 0.9987900853157043,
            "predictions": {
                "draw-circle": 1.553169681756117e-08,
                "draw-cross": 1.0978459807908791e-10,
                "fever": 0.9987900853157043,
                "floss": 2.618965481815394e-05,
                "roll": 9.422032043460149e-09,
                "shake": 0.001183744752779603
            }
        }
    ],
    "total": 1
}


### Fake

In [11]:
fake_instances = data_dance_raw['floss'][-10:]

[d.update(gesture='roll') for d in fake_instances]

data = {
    "instances": fake_instances
}
json_response = requests.post("http://localhost:5000/model/predict", json=data, headers={"Host": 'localhost', "Content-Type":"application/json"})

print(json.dumps(json_response.json(), indent=4))

{
    "payload": [
        {
            "candidate": "fever",
            "candidate_score": 0.8268402814865112,
            "predictions": {
                "draw-circle": 0.00029087907751090825,
                "draw-cross": 5.402942861110205e-06,
                "fever": 0.8268402814865112,
                "floss": 0.17253518104553223,
                "roll": 0.0002907082380261272,
                "shake": 3.753391501959413e-05
            }
        },
        {
            "candidate": "roll",
            "candidate_score": 0.7174426913261414,
            "predictions": {
                "draw-circle": 0.00213981862179935,
                "draw-cross": 3.92303434182395e-07,
                "fever": 0.2431594878435135,
                "floss": 0.03725210949778557,
                "roll": 0.7174426913261414,
                "shake": 5.46865430806065e-06
            }
        },
        {
            "candidate": "roll",
            "candidate_score": 0.952195942401886,
            "

### Knative

In [ ]:
%%bash

oc start-build tf-serving --from-dir .

In [ ]:
builds = !oc get builds
latest = builds[-1].split()[0]

!oc logs -f build.build.openshift.io/$latest

In [ ]:
revision = int(dt.timestamp(dt.now()))

!oc process tf-serving-knative -p REVISION=$revision | oc apply -f -

In [ ]:
%%bash

while oc get pods | grep tf-serving-knative | grep -v Running; do 
    oc get pods | grep tf-serving-knative
    sleep 1
done

In [ ]:
%%bash --out KSERVICE_DOMAIN

oc get kservice tf-serving-knative-demo -o jsonpath='{.status.domain}'

In [ ]:
KSERVICE_DOMAIN

In [ ]:
import requests

INGRESS_GATEWAY="istio-ingressgateway.istio-system.svc.cluster.local"

data = {
  "instances": [
      json.loads(Path('data/dance/floss/example.json').read_text())
  ]
}
json_response = requests.post("http://%s/model/predict" % INGRESS_GATEWAY, json=data, headers={"Host": KSERVICE_DOMAIN, "Content-Type":"application/json"})

print(json.dumps(json_response.json(), indent=4))